In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import operator
import numpy as np
import matplotlib.pyplot as plt
import logging

In [2]:
def make_plot(counts):
    """
    This function plots the counts of positive and negative words for each timestep.
    """
    positiveCounts = []
    negativeCounts = []
    time = []
    
    
    for val in counts:
        print(val)
        positiveTuple = val[0]
        positiveCounts.append(positiveTuple[1])
        negativeTuple = val[1]
        negativeCounts.append(negativeTuple[1])

    for i in range(len(counts)):
        time.append(i)

    posLine = plt.plot(time, positiveCounts, 'bo-', label='Positive')
    negLine = plt.plot(time, negativeCounts, 'go-', label='Negative')
    plt.axis([0, len(counts), 0, max(max(positiveCounts), max(negativeCounts)) + 50])
    plt.xlabel('Time step')
    plt.ylabel('Word count')
    plt.legend(loc='upper left')
    plt.show()

In [3]:
def load_wordlist(filename):
    """
    This function returns a list or set of words from the given filename.
    """
    words = {}
    f = open(filename, 'r')
    text = f.read()
    text2 = text.split('\n')
    for line in text2:
        words[line] = 1
    f.close()
    return words


def wordSentiment(word, pwords, nwords):
    if word in pwords:
        return ('positive', 1)
    elif word in nwords:
        return ('negative', 1)


def updateFunction(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
    return sum(newValues, runningCount)


def sendRecord(record):
    connection = createNewConnection()
    connection.send(record)
    connection.close()

In [4]:
def stream(ssc, pwords, nwords, duration):
    kstream = KafkaUtils.createDirectStream(
        ssc, topics=["tweets"], kafkaParams={"metadata.broker.list": 'localhost:9092'})
    tweets = kstream.map(lambda x: x[1])

    # Each element of tweets will be the text of a tweet.
    # We keep track of a running total counts and print it at every time step.
    words = tweets.flatMap(lambda line: line.split(" "))
    positive = words.map(lambda word: ('Positive', 1) if word in pwords else ('Positive', 0))
    negative = words.map(lambda word: ('Negative', 1) if word in nwords else ('Negative', 0))
    allSentiments = positive.union(negative)
    sentimentCounts = allSentiments.reduceByKey(lambda x, y: x + y)
    runningSentimentCounts = sentimentCounts.updateStateByKey(updateFunction)
    runningSentimentCounts.pprint()

    # The counts variable hold the word counts for all time steps
    counts = []
    sentimentCounts.foreachRDD(lambda t, rdd: counts.append(rdd.collect()))
    #logger = logging.getLogger()
    #logger.setLevel(logging.DEBUG)
    #logging.debug(counts)
    # Start the computation
    ssc.start()
    ssc.awaitTerminationOrTimeout(duration)
    ssc.stop(stopGraceFully=True)

    return counts

In [5]:
sc.stop()

In [6]:
conf = SparkConf().setMaster("local[2]").setAppName("Streamer")
sc = SparkContext(conf=conf)

    # Creating a streaming context with batch interval of 10 sec
ssc = StreamingContext(sc, 10)
ssc.checkpoint("checkpoint")
pwords = load_wordlist("positive-words.txt")
nwords = load_wordlist("negative-words.txt")

In [7]:
counts = stream(ssc, pwords, nwords, 1000)
make_plot(counts)

-------------------------------------------
Time: 2017-12-03 22:02:30
-------------------------------------------
('Positive', 349)
('Negative', 99)

-------------------------------------------
Time: 2017-12-03 22:02:40
-------------------------------------------
('Positive', 1268)
('Negative', 316)

-------------------------------------------
Time: 2017-12-03 22:02:50
-------------------------------------------
('Positive', 2337)
('Negative', 530)

-------------------------------------------
Time: 2017-12-03 22:03:00
-------------------------------------------
('Positive', 3269)
('Negative', 744)

-------------------------------------------
Time: 2017-12-03 22:03:10
-------------------------------------------
('Positive', 4200)
('Negative', 954)

-------------------------------------------
Time: 2017-12-03 22:03:20
-------------------------------------------
('Positive', 5073)
('Negative', 1162)

-------------------------------------------
Time: 2017-12-03 22:03:30
-------------------

-------------------------------------------
Time: 2017-12-03 22:11:30
-------------------------------------------
('Positive', 11563)
('Negative', 2689)

-------------------------------------------
Time: 2017-12-03 22:11:40
-------------------------------------------
('Positive', 11563)
('Negative', 2689)

-------------------------------------------
Time: 2017-12-03 22:11:50
-------------------------------------------
('Positive', 11563)
('Negative', 2689)

-------------------------------------------
Time: 2017-12-03 22:12:00
-------------------------------------------
('Positive', 11563)
('Negative', 2689)

-------------------------------------------
Time: 2017-12-03 22:12:10
-------------------------------------------
('Positive', 11563)
('Negative', 2689)

-------------------------------------------
Time: 2017-12-03 22:12:20
-------------------------------------------
('Positive', 11563)
('Negative', 2689)

-------------------------------------------
Time: 2017-12-03 22:12:30
------

Py4JJavaError: An error occurred while calling o58.awaitTerminationOrTimeout.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\streaming\util.py", line 65, in call
    r = self.func(t, *rdds)
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\streaming\dstream.py", line 171, in takeAndPrint
    taken = rdd.take(num + 1)
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\rdd.py", line 1343, in take
    res = self.context.runJob(self, takeUpToNumLeft, p)
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\pyspark\context.py", line 992, in runJob
    port = self._jvm.PythonRDD.runJob(self._jsc.sc(), mappedRDD._jrdd, partitions)
  File "C:\opt\spark\spark-2.2.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip\py4j\java_gateway.py", line 1133, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "C:\opt\spark\spark-2.2.0-bin-hado